# **Face Mask Classification** 😷

## **0.** Settings

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Unzip
!unzip '/content/drive/MyDrive/Face Mask/dataset.zip'

# Get the directory
cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'dataset')

In [3]:
# Libraries
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from IPython.display import Image, display
from tensorflow.keras import regularizers
from tensorflow import keras
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
import seaborn as sns
import numpy as np
import math
import cv2

# Warnings off
import warnings
warnings.filterwarnings("ignore")

# Split in train and validation
img_h = 256
img_w = 256
batch_size = 32

train_dataset = image_dataset_from_directory(
    dataset_dir,
    shuffle          = True,
    batch_size       = batch_size,
    image_size       = (img_h, img_w),
    validation_split = 0.3,
    subset           = 'training',
    seed             = 420)

validation_dataset = image_dataset_from_directory(
    dataset_dir,
    shuffle          = True,
    batch_size       = batch_size,
    image_size       = (img_h, img_w),
    validation_split = 0.3,
    subset           = 'validation',
    seed             = 420)

classes = train_dataset.class_names
num_classes = len(classes)

Found 5614 files belonging to 3 classes.
Using 3930 files for training.
Found 5614 files belonging to 3 classes.
Using 1684 files for validation.


## **1.** Single-model predictions

In [ ]:
from tqdm.auto import tqdm
import pandas as pd

## We need to manually go and get the validation images (which are the last 30% of the dataset)

model_names = ['model_ensamble_01.hdf5', 
               'model_ensamble_02.hdf5', 
               'model_ensamble_03.hdf5', 
               'model_ensamble_04.hdf5',
               'model_ensamble_05.hdf5', 
               'model_ensamble_06.hdf5', 
               'model_ensamble_07.hdf5']

for model_name in tqdm(model_names, position=0, leave=True):
    
    print('Start with: ', model_name)
    model = tf.keras.models.load_model('/content/drive/MyDrive/Face Mask/' + model_name)

    y_true = []
    y_pred = []
    
    #-------------------------------------------------
    # Label 0
    #-------------------------------------------------
    count = 0
    print('Label 0')
    for i in tqdm(os.listdir(dataset_dir + '/0'), position=0, leave=True):
      count += 1
      if count > 1000 :  
          y_true.append(0)
          img = image.load_img(dataset_dir + '/0' + '/' + i, target_size=(img_h, img_w))
          X = image.img_to_array(img)
          X = np.expand_dims(X, axis=0)
          images = np.vstack([X])
          y_pred.append(np.argmax(model.predict(images)))

    #-------------------------------------------------
    # Label 1
    #-------------------------------------------------
    count = 0
    print('Label 1')
    for i in tqdm(os.listdir(dataset_dir + '/1'), position=0, leave=True):
      count += 1
      if count > 1000 :  
        y_true.append(1)
        img = image.load_img(dataset_dir + '/1' + '/' + i, target_size=(img_h, img_w))
        X = image.img_to_array(img)
        X = np.expand_dims(X, axis=0)
        images = np.vstack([X])
        y_pred.append(np.argmax(model.predict(images)))

    #-------------------------------------------------
    # Label 2
    #-------------------------------------------------
    print('Label 2')
    count = 0
    for i in tqdm(os.listdir(dataset_dir + '/2'), position=0, leave=True):
      count += 1
      if count > 1000 :  
        y_true.append(2)
        img = image.load_img(dataset_dir + '/2' + '/' + i, target_size=(img_h, img_w))
        X = image.img_to_array(img)
        X = np.expand_dims(X, axis=0)
        images = np.vstack([X])
        y_pred.append(np.argmax(model.predict(images)))

    if model_name == 'model_ensamble_01.hdf5':
        records = pd.DataFrame(y_true, columns=['true_labels'])
    
    records[model_name] = y_pred

## **2.** Ensamble prediction

In [50]:
records.T

,0,1,2,3,4,5,6,7,8,9,...,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613
true_labels,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
model_ensamble_01.hdf5,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
model_ensamble_02.hdf5,0,0,0,0,0,0,0,0,0,0,...,1,1,2,2,2,2,2,1,2,2
model_ensamble_03.hdf5,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
model_ensamble_04.hdf5,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
model_ensamble_05.hdf5,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
model_ensamble_06.hdf5,0,0,0,0,2,0,0,0,0,0,...,2,2,1,2,2,1,2,1,2,2
model_ensamble_07.hdf5,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,1,2,1,2,2


In [49]:
print('Model 1 - Accuracy: ', np.count_nonzero(records['true_labels'] == records['model_ensamble_01.hdf5'])/len(records['true_labels']))
print('Model 2 - Accuracy: ', np.count_nonzero(records['true_labels'] == records['model_ensamble_02.hdf5'])/len(records['true_labels']))
print('Model 3 - Accuracy: ', np.count_nonzero(records['true_labels'] == records['model_ensamble_03.hdf5'])/len(records['true_labels']))
print('Model 4 - Accuracy: ', np.count_nonzero(records['true_labels'] == records['model_ensamble_04.hdf5'])/len(records['true_labels']))
print('Model 5 - Accuracy: ', np.count_nonzero(records['true_labels'] == records['model_ensamble_05.hdf5'])/len(records['true_labels']))
print('Model 6 - Accuracy: ', np.count_nonzero(records['true_labels'] == records['model_ensamble_06.hdf5'])/len(records['true_labels']))
print('Model 7 - Accuracy: ', np.count_nonzero(records['true_labels'] == records['model_ensamble_07.hdf5'])/len(records['true_labels']))

Model 1 - Accuracy:  0.9747513389441469
Model 2 - Accuracy:  0.8672532517214996
Model 3 - Accuracy:  0.9322876817138485
Model 4 - Accuracy:  0.9636572302983932
Model 5 - Accuracy:  0.9651874521805662
Model 6 - Accuracy:  0.9296097934200459
Model 7 - Accuracy:  0.9452945677123182


In [66]:
ensemble_prediction = []

for i in range(len(records)):
    
    models_predictions = np.array(records.iloc[i][1:8])

    # Append most frequent element
    ensemble_prediction.append(np.bincount(models_predictions).argmax())

In [68]:
print('Ensemble - Accuracy: ', np.count_nonzero(records['true_labels'] == ensemble_prediction)/len(records['true_labels']))

Ensemble - Accuracy:  0.9747513389441469
